In [1]:
"""
https://statkclee.github.io/ml/ml-pm-continuous.html
https://patentimages.storage.googleapis.com/18/88/18/9866f5716b47f6/KR20000036594A.pdf
https://datascienceschool.net/view-notebook/8d086f61e13142aab9bea812f986d0fe/
http://contents.kocw.or.kr/KOCW/document/2014/Chungbuk/choisanghyun/7.pdf
https://github.com/jaisenc/carstats
https://github.com/hsteude/used_car_prices/blob/master/model_training.py
https://github.com/quangbk2010/UsedCar/blob/master/Code/Neural_network_tensorflow.py

https://www.kaggle.com/ironfrown/deep-learning-house-price-prediction-keras
http://androidkt.com/car-price-prediction/
==================================================================================================
Used vehicle price estimator with TF/Keras
We don't know what data will be provided for this task now(190314).
So, Would rather prepare all possible, imaginable scinarios that may happen in the future.

Look of data that would be provided I assume...
Structured format : used vehicle transaction data(history).
car information
buyer's information - 고객특성, 고객 퍼포먼스-sequential 임베딩(LSTM, GRU등)
seller's information
dealer's information
other information?

Unstructured format : 
used vehicle images 이미지 CNN응용(VGG또는 Googlnet에서 featuremap 추출)
-> extract features from pre-trained vggnet and make token of each image -> average -> merge to base-df
used vehicle seller/dealer's comment  LSTM, Attentionnet등을 통해 featuremap 추출
-> extract features from seq_embedding, fasttext, w2v, d2v, attention, pre-trained models -> merge to base-df
모두 concatenate
network 구성 relu,relu,relu......-> softmax -> 평가
"""

"\nhttps://statkclee.github.io/ml/ml-pm-continuous.html\nhttps://patentimages.storage.googleapis.com/18/88/18/9866f5716b47f6/KR20000036594A.pdf\nhttps://datascienceschool.net/view-notebook/8d086f61e13142aab9bea812f986d0fe/\nhttp://contents.kocw.or.kr/KOCW/document/2014/Chungbuk/choisanghyun/7.pdf\nhttps://github.com/jaisenc/carstats\nhttps://github.com/hsteude/used_car_prices/blob/master/model_training.py\nhttps://github.com/quangbk2010/UsedCar/blob/master/Code/Neural_network_tensorflow.py\n\nhttps://www.kaggle.com/ironfrown/deep-learning-house-price-prediction-keras\nhttp://androidkt.com/car-price-prediction/\n==================================================================================================\nUsed vehicle price estimator with TF/Keras\nWe don't know what data will be provided for this task now(190314).\nSo, Would rather prepare all possible, imaginable scinarios that may happen in the future.\n\nLook of data that would be provided I assume...\nStructured format : used 

In [1]:
# global import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# preprocessing
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
# models
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR, LinearSVR
from sklearn.ensemble import AdaBoostRegressor, BaggingRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor

%matplotlib inline

In [2]:
import pickle

In [ ]:
with open("./Datasets/state_province.pkl",'wb') as f:
    pickle.dump(state_province_dictionary, f)
    
with open("./Datasets/state_province.pkl",'rb') as f:
    state_province_dictionary = pickle.load(f)

In [8]:
with open('./ToyotaCorolla.pkl','wb') as a:
    pickle.dump(df, a)

In [3]:
with open('./Datasets/ToyotaCorolla.pkl','rb') as a:
    cc = pickle.load(a)

In [4]:
cc

Price  Age     KM FuelType   HP  MetColor  Automatic    CC  Doors  \
0     13500   23  46986   Diesel   90         1          0  2000      3   
1     13750   23  72937   Diesel   90         1          0  2000      3   
2     13950   24  41711   Diesel   90         1          0  2000      3   
3     14950   26  48000   Diesel   90         0          0  2000      3   
4     13750   30  38500   Diesel   90         0          0  2000      3   
5     12950   32  61000   Diesel   90         0          0  2000      3   
6     16900   27  94612   Diesel   90         1          0  2000      3   
7     18600   30  75889   Diesel   90         1          0  2000      3   
8     21500   27  19700   Petrol  192         0          0  1800      3   
9     12950   23  71138   Diesel   69         0          0  1900      3   
10    20950   25  31461   Petrol  192         0          0  1800      3   
11    19950   22  43610   Petrol  192         0          0  1800      3   
12    19600   25  32189   Petrol  192         0          0  1800      3   
13    21500   31  23000   Petrol  192         1          0  1800      3   
14    22500   32  34131   Petrol  192         1          0  1800      3   
15    22000   28  18739   Petrol  192         0          0  1800      3   
16    22750   30  34000   Petrol  192         1          0  1800      3   
17    17950   24  21716   Petrol  110         1          0  1600      3   
18    16750   24  25563   Petrol  110         0          0  1600      3   
19    16950   30  64359   Petrol  110         1          0  1600      3   
20    15950   30  67660   Petrol  110         1          0  1600      3   
21    16950   29  43905   Petrol  110         0          1  1600      3   
22    15950   28  56349   Petrol  110         1          0  1600      3   
23    16950   28  32220   Petrol  110         1          0  1600      3   
24    16250   29  25813   Petrol  110         1          0  1600      3   
25    15950   25  28450   Petrol  110         1          0  1600      3   
26    17495   27  34545   Petrol  110         1          0  1600      3   
27    15750   29  41415   Petrol  110         1          0  1600      3   
28    16950   28  44142   Petrol  110         0          0  1600      3   
29    17950   30  11090   Petrol  110         1          0  1600      3   
...     ...  ...    ...      ...  ...       ...        ...   ...    ...   
1406   8950   70  44850   Petrol  110         1          0  1600      3   
1407   8250   69  44826   Petrol  110         0          0  1600      5   
1408   9250   80  44444   Petrol  110         1          0  1600      3   
1409   7900   75  43720   Petrol  110         1          0  1600      5   
1410   8500   78  43622   Petrol   86         1          0  1300      4   
1411   7950   76  43532   Petrol  110         0          0  1600      5   
1412   9950   69  42800   Petrol  110         1          0  1600      3   
1413   8750   74  42317   Petrol  107         1          1  1600      5   
1414   7500   80  42186   Petrol  110         1          0  1600      3   
1415   6950   72  42000   Petrol  110         1          0  1600      3   
1416   8950   79  40093   Petrol  110         0          0  1600      5   
1417   8750   79  39800   Petrol  107         0          1  1600      3   
1418   7750   73  39168   Petrol   86         0          0  1300      3   
1419   8450   75  38945   Petrol  110         1          0  1600      3   
1420   8150   76  36537   Petrol  110         0          1  1600      4   
1421   8500   78  36000   Petrol   86         0          1  1300      3   
1422   7600   78  36000   Petrol  110         1          0  1600      3   
1423   7950   80  35821   Petrol   86         0          1  1300      3   
1424   7750   73  34717   Petrol   86         0          0  1300      3   
1425   7950   80  34000   Petrol   86         1          0  1300      4   
1426   9950   78  30964   Petrol  110         0          1  1600      3   
1427   8950   71  29000   Petro

In [6]:
df = pd.read_csv("./Datasets/ToyotaCorolla.csv")

In [7]:
df.head(2)

Price  Age     KM FuelType  HP  MetColor  Automatic    CC  Doors  Weight
0  13500   23  46986   Diesel  90         1          0  2000      3    1165
1  13750   23  72937   Diesel  90         1          0  2000      3    1165

In [ ]:
df['Automatic'] = df.Automatic.astype(bool)
df['MetColor'] = df.MetColor.astype(bool)

# Price 값의 좌측으로 기울었음. log변환
# plt.hist(df.Price.sort_values())
# plt.show()

df['Price'] = np.log(df.Price)

In [3]:
lb = LabelBinarizer()
df = df.join(pd.DataFrame(lb.fit_transform(df.FuelType))).drop('FuelType', axis=1)

NameError: name 'df' is not defined

In [34]:
train, nottrain = train_test_split(df, test_size= 0.4)
valid, test = train_test_split(nottrain, test_size= 0.5)

In [ ]:
# 이미지 처리 시나리오
# 중고차 A의 각 부분에 대한 K개의 이미지가 있다고 가정하자.(K개는 VARLENFEATURE이다)
# K개의 이미지의 임베딩 벡터를 VGGnet으로 추출한다.
# K개의 이미지가 K개의 벡터로 표현되었으면 이들의 평균을 취하여 중고차 A의 representation으로 사용한다.

In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
import tensornets as nets # CNN, Object detection
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
import numpy as np
import cv2, os

In [3]:
filedir = "./Datasets"
files = ['Basketball.jpeg','chair.jpg','Koala.jpg','Koala2.jpg']

In [4]:
for i in files:
    img=nets.utils.load_img(os.path.join(filedir, i), target_size=256, crop_size=224)
    img=cv2.normalize(img, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    if i == files[0]:
        batch_img = img
    else:
        batch_img = np.concatenate([batch_img, img])

In [5]:
batch_img.shape

(4, 224, 224, 3)

### https://github.com/machrisaa/tensorflow-vgg ????

### 1.[keras](https://keras.io/applications/)에서 제공하는 resnet50을 활용

In [25]:
# model = ResNet50(weights='imagenet', include_top=False)
# # model.summary()

# model.compile(loss = 'categorical_crossentropy', optimizer='adadelta', metrics = ['accuracy'])
# yhat = model.predict(x=batch_img, batch_size=4)

# aa = model.predict_on_batch(x=batch_img)
# aa.shape

# label = decode_predictions(yhat) # aws접속 불가로 에러남
# label = label[0][0]
# print(label)
# 중간레이어 feature 추출가능, 파인튜닝가능

### 2.[tensornets](https://github.com/taehoonlee/tensornets)를 활용

In [7]:
inputs = tf.placeholder(tf.float32, [None, 224, 224, 3])
model = nets.ResNet50(inputs)

assert isinstance(model, tf.Tensor)
model.print_summary()

Scope: resnet50
Total layers: 54
Total weights: 320
Total parameters: 25,636,712


In [8]:
with tf.Session() as sess:
    processed_img = model.preprocess(batch_img)  # equivalent to img = nets.preprocess(model, img)
    sess.run(model.pretrained())  # equivalent to nets.pretrained(model)
    preds = sess.run(model, {inputs: processed_img})
    middles = sess.run(model.get_middles(), {inputs: processed_img})
    outputs = sess.run(model.get_outputs(), {inputs: processed_img})

In [22]:
# print(nets.utils.decode_predictions(preds, top=2)[0]) # aws접속 불가로 에러남 -- json파일 다운받아서 직접 시도(아래)
# [print(middles[ii].shape) for ii in range(len(middles))] # middle layer features
# [print(outputs[ii].shape) for ii in range(len(outputs))] # all layer features

showmetop = 1
with open('./Datasets/imagenet_class_index.json') as data_file:    
    import json
    data = json.load(data_file)
    
results=[]
for pred in preds:
    top_indices = pred.argsort()[-showmetop:][::-1]
    result = [tuple(data[str(i)]) + (pred[i],) for i in top_indices]
    result.sort(key=lambda x: x[2], reverse=True)
    results.append(result)
    
results

[[('n06359193', 'web_site', 0.06514467)],
 [('n03196217', 'digital_clock', 0.05937307)],
 [('n06359193', 'web_site', 0.06149534)],
 [('n06359193', 'web_site', 0.05729807)]]

In [47]:
# final - bottelneck layer's feature vector 
btlr = middles[len(middles)-1].reshape(4,-1)
btlr

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
# # networks comparison
# inputs = tf.placeholder(tf.float32, [None, 224, 224, 3])
# models = [
#     nets.MobileNet75(inputs),
#     nets.MobileNet100(inputs),
#     nets.SqueezeNet(inputs),
# ]

# img = utils.load_img('cat.png', target_size=256, crop_size=224)
# imgs = nets.preprocess(models, img)

# with tf.Session() as sess:
#     nets.pretrained(models)
#     for (model, img) in zip(models, imgs):
#         preds = sess.run(model, {inputs: img})
#         print(utils.decode_predictions(preds, top=2)[0])

In [ ]:
# # Transfer learning:
# inputs = tf.placeholder(tf.float32, [None, 224, 224, 3])
# outputs = tf.placeholder(tf.float32, [None, 50])
# model = nets.DenseNet169(inputs, is_training=True, classes=50)

# loss = tf.losses.softmax_cross_entropy(outputs, model.logits)
# train = tf.train.AdamOptimizer(learning_rate=1e-5).minimize(loss)

# with tf.Session() as sess:
#     nets.pretrained(model)
#     for (x, y) in your_NumPy_data:  # the NHWC and one-hot format
#         sess.run(train, {inputs: x, outputs: y})

In [ ]:
# # Using multi-GPU
# inputs = tf.placeholder(tf.float32, [None, 224, 224, 3])
# models = []

# with tf.device('gpu:0'):
#     models.append(nets.ResNeXt50(inputs))

# with tf.device('gpu:1'):
#     models.append(nets.DenseNet201(inputs))

# from tensornets.preprocess import fb_preprocess
# img = utils.load_img('cat.png', target_size=256, crop_size=224)
# img = fb_preprocess(img)

# with tf.Session() as sess:
#     nets.pretrained(models)
#     preds = sess.run(models, {inputs: img})
#     for pred in preds:
#         print(utils.decode_predictions(pred, top=2)[0])

In [ ]:
# 텍스트 처리 시나리오
# 중고차 A의 소개글이 있다고 가정하자.
# A의 소개글을 Tokenizing 한 뒤 
# WORD2VEC or fasttext -> WORD VECTOR들의 평균으로 소개글의 representation으로 사용한다.
# DOC2VEC -> 소개글의 representation으로 사용한다.

In [ ]:
df = pd.DataFrame(pd.Series(list(vehicle[["INT_COLOR_CD", "EXT_COLOR_CD"]].values))).reset_index()
df.columns = ['docID','item']
df1 = [((row['item']), row['docID']) for idx, row in df.iterrows()]
df2 = namedtuple('TaggedDocument', 'words tags')
tagged_data = [df2(d, [c]) for d, c in df1]

from gensim.models import Doc2Vec
model = Doc2Vec(dm=0, dbow_words=0, window =2, vector_size=10, alpha=0.025, min_alpha=0.025, seed=0, sample=1e-5, min_count=2, workers=multiprocessing.cpu_count(), 
        hs = 0, negative = 10)
model.build_vocab(tagged_data)
model.train(documents=tagged_data, total_examples=df.shape[0], epochs=10)
model.save("./vehicle_color_embedding.d2vmodel")
print("---done---")
model=Doc2Vec.load("./vehicle_color_embedding.d2vmodel")
embedding_df = pd.DataFrame(
            data = [model.infer_vector(doc.words) for doc in tagged_data], 
            columns = ["d2v"+str(i) for i in range(10)])

In [ ]:
# 피쳐를 다 뽑으면 Structured data에 merge한 후...

In [ ]:
categorical_inputs = {}
sequential_inputs = {}
categorical_embeddings = {}
sequential_embeddings = {}
embedding_size_category = 10
embedding_size_sequence = 50
sequence_layer_list = []
input_layer_list = []

# feature column processing
for column in categorical_columns:
    categorical_inputs[column] = Input(shape=[1],name=column)
    input_layer_list.append(categorical_inputs[column])
    categorical_embeddings[column] = (Embedding(max_dict[column],
                                                embedding_size_category)(categorical_inputs[column]))
    categorical_embeddings[column] = (BatchNormalization()(categorical_embeddings[column]))
    categorical_embeddings[column] = Dropout(dropout)(Flatten()(categorical_embeddings[column]))

for column in sequential_columns:
    sequential_inputs[column] = Input(shape=[X_train[column].shape[1]],name=column)
    input_layer_list.append(sequential_inputs[column])
    sequential_embeddings[column] = (Embedding(max_sequence_for_embeds[column],
                                               embedding_size_sequence)(sequential_inputs[column]))
    sequential_embeddings[column] = (BatchNormalization()(sequential_embeddings[column])) 
    sequential_embeddings[column] = Dropout(dropout)(GRU(16,kernel_regularizer=l2(l2_lambda))(sequential_embeddings[column]))

main_l = sequential_embeddings['X_TRANSMISSION_DESC']
main_l = concatenate([main_l, sequential_embeddings['X_MODEL_DESCRIPTION']])
# concatenate(axis=-1), 디폴트로 하면 컬럼와이즈하게 인풋이 붙는다.
for column in categorical_embeddings.keys():
    main_l = concatenate([main_l, categorical_embeddings[column]])

print("list of layers:", main_l) 

output = Dense(1, activation="sigmoid")(main_l)
model = Model(input_layer_list, output)
optimizer = Adam(lr)
model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

In [ ]:
# BATCH_SIZE = 2000
# epochs = 10
print("Learning rate ",lr)
print("dropout ",dropout)
print("L2 lambda ",l2_lambda)
print("batch size ",batch_size)
print("epochs ",epochs)

model = get_model()
model.summary()
modelfit = model.fit(X_train, training.y, epochs=epochs, batch_size=batch_size
          , validation_data=(X_eval, evaluation.y), verbose=1)

## 목적---
#### 인공지능 기반의 중고차 시세 조회
#### 고객 분석을 바탕으로 맞춤형 정보제공
#### 데이터 시각화등을 통한 실감형 콘텐츠 제공
## 예상되는 Y값---
#### 예상시세, 시세 트렌드, 수요와 공급의 영향에 맞춰 동적인 예상시세를 산출한다.
#### 고객 분석 -> 정보 추천 또 wnd일것인가? 계속 변화하는 y 값일 것.